Import statements

In [33]:
import requests    # http://python-requests.org/
import json
import time
from collections import Counter

Create initial lists/dictionaries. **Thank you [Master League](https://masterleague.net/) for the data!**

In [ ]:
endpoints = {
    "heroes": "https://api.masterleague.net/heroes/?format=api",
    "maps": "https://api.masterleague.net/maps/?format=api",
    "regions": "https://api.masterleague.net/regions/?format=api",
    "patches": "https://api.masterleague.net/patches/?format=api",
    "teams": "https://api.masterleague.net/teams/?format=api",
    "players": "https://api.masterleague.net/players/?format=api",
    "tournaments": "https://api.masterleague.net/tournaments/?format=api",
    "matches": "https://api.masterleague.net/matches/?format=api",
    "calendar": "https://api.masterleague.net/calendar/?format=api"
}

How to use the API for unauthenticated users. https://api.masterleague.net/?format=api

In [2]:
API_ROOT = 'https://api.masterleague.net'
API_USER = None
API_PASS = None

session = requests.Session()

Get data from heroes API.

In [3]:
r = session.get(API_ROOT + '/heroes.json')
data = json.loads(r.text)
heroes = {}
next_page = data['next']

for hero in data['results']:
    heroes[hero['id']] = hero['name']

while data['next'] is not None:
    next_page = data['next']
    text_results = session.get(next_page)
    data = json.loads(text_results.text)
    
    next_page = data['next']
    for hero in data['results']:
        heroes[hero['id']] = hero['name']

Test call to match API

In [ ]:
match_response = session.get(API_ROOT + '/matches.json')
match_data = json.loads(match_response.text)
match_data

Test call to tournament API

In [ ]:
tournament_response = session.get(API_ROOT + '/tournaments.json')
tournament_data = json.loads(tournament_response.text)
#tournament_data

## Tournament Functions

In [4]:
def add_tournament_details(tournament_dictionary, page_data):
    for tournament in page_data:
        tournament_id = tournament['id']
        tournament_name = tournament['name']
        tournament_dictionary[tournament_id] = tournament_name
    return tournament_dictionary

def create_tournament_dictionary(api_session, API_ROOT):
    tournaments = {}
    next_tournament_url = API_ROOT + '/tournaments.json'

    while next_tournament_url is not None:
        text_results = api_session.get(next_tournament_url)
        tournament_data = json.loads(text_results.text)
        page_data = tournament_data['results']
        tournaments = add_tournament_details(tournaments, page_data)
        next_tournament_url = tournament_data['next']
    
    return tournaments
    

## Match Functions

In [5]:
def get_match_data(tournament_id, API_ROOT):
    session = requests.Session()
    matches = {}
    match_url = API_ROOT + '/matches.json'
    
    while match_url is not None:
        text_results = session.get(match_url)
        match_data = json.loads(text_results.text)
        #print(match_data)
        for match in match_data['results']:
            tournament = match['tournament']
            if tournament == tournament_id:
                match_id = match['id']
                matches[match_id] = match
        time.sleep(2)
        match_url = match_data['next']
        
    return matches
    

### Create tournament dictionary

In [6]:
API_ROOT = 'https://api.masterleague.net'
API_USER = None
API_PASS = None

session = requests.Session()

In [7]:
tournament_dict = create_tournament_dictionary(session, API_ROOT)

In [ ]:
tournament_dict

https://api.masterleague.net/?format=api

Get match data for match **32: HGC North America - Phase 1**

In [8]:
match_dict = get_match_data(32, API_ROOT)

In [2]:
#match_dict

sleep 5 seconds, more than 45 minutes to run. what is the minimum time we can sleep?
Start: 11:46 (sleep 1 second) - wrong. infinite loop.
Restarted: 11:52 (sleep 1 second) - got results error. add printout of error message (try/catch?)

Started at 6:49 PM. sleep 2 seconds. Still running at 6:59. Finished at 7:04 - 15 min.

download pickle. import pickle. pickle the match dictionary ASAP.

In [1]:
import pickle
#pickle.dump( match_dict, open( "tournament_32_matches.p", "wb" ) )
match_dict = pickle.load(open('tournament_32_matches.p', 'rb'))

Which fields are relevant? id, drafts['is_winner'], drafts[0-1]['picks'][0-4]['hero'], map, patch

In [2]:
relevant_data = {}

for item in match_dict:
    match = match_dict.get(item)
    #print(match)
    match_id = match['id']
    draft_info = match['drafts']
    #draft_1 = match['drafts'][0]
    #draft_2 = match['drafts'][1]
    match_map = match['map']
    patch = match['patch']
    #print("id: %d, draft: %s, map: %s, patch: %d" % (match_id, draft_info, match_map, patch))
    relevant_data[match_id] = [patch, match_map, draft_info]
    


In [3]:
relevant_data

{1762: [14,
  10,
  [{'bans': [21, 51],
    'is_winner': True,
    'picks': [{'hero': 27, 'player': 144},
     {'hero': 55, 'player': 174},
     {'hero': 2, 'player': 176},
     {'hero': 15, 'player': 490},
     {'hero': 18, 'player': 194}],
    'team': 108},
   {'bans': [50, 57],
    'is_winner': False,
    'picks': [{'hero': 23, 'player': 485},
     {'hero': 33, 'player': 384},
     {'hero': 48, 'player': 380},
     {'hero': 14, 'player': 175},
     {'hero': 47, 'player': 382}],
    'team': 129}]],
 1763: [14,
  11,
  [{'bans': [42, 27],
    'is_winner': True,
    'picks': [{'hero': 50, 'player': 144},
     {'hero': 16, 'player': 490},
     {'hero': 54, 'player': 194},
     {'hero': 32, 'player': 174},
     {'hero': 2, 'player': 176}],
    'team': 108},
   {'bans': [23, 55],
    'is_winner': False,
    'picks': [{'hero': 21, 'player': 382},
     {'hero': 10, 'player': 485},
     {'hero': 53, 'player': 384},
     {'hero': 34, 'player': 175},
     {'hero': 7, 'player': 380}],
    'team

Now we have HGC NA Phase 1 - all match data.

In [4]:
len(relevant_data)

274

create dictionary where each key is the match id, and the value is a tuple of 2 sorted lists: the winning team comp and the losing team comp.

In [44]:
comps = {}

for match in relevant_data:
    winning_team = []
    losing_team = []
    #print(match)
    if relevant_data[match][2][0]['is_winner'] is True:        
        for hero in relevant_data[match][2][0]['picks']:
            #hero_num = 
            winning_team.append(hero['hero'])
        for hero in relevant_data[match][2][1]['picks']:
            losing_team.append(hero['hero'])
    else:
        for hero in relevant_data[match][2][1]['picks']:
            winning_team.append(hero['hero'])
        for hero in relevant_data[match][2][0]['picks']:
            losing_team.append(hero['hero'])
            
    comp = (tuple(sorted(winning_team)), tuple(sorted(losing_team)))
    comps[match] = comp
    

In [45]:
comps

{1762: ((2, 15, 18, 27, 55), (14, 23, 33, 47, 48)),
 1763: ((2, 16, 32, 50, 54), (7, 10, 21, 34, 53)),
 1764: ((14, 21, 27, 30, 33), (20, 22, 26, 32, 48)),
 1765: ((16, 23, 28, 32, 33), (21, 31, 46, 48, 50)),
 1766: ((21, 26, 28, 32, 33), (23, 27, 30, 31, 48)),
 1767: ((12, 27, 33, 55, 57), (16, 23, 32, 50, 54)),
 1768: ((21, 27, 34, 41, 54), (2, 11, 20, 47, 48)),
 1769: ((16, 21, 30, 42, 48), (14, 23, 26, 56, 57)),
 1770: ((7, 21, 24, 32, 56), (2, 10, 14, 27, 33)),
 1771: ((2, 16, 41, 42, 48), (3, 21, 26, 32, 50)),
 1772: ((16, 23, 24, 42, 54), (2, 4, 27, 30, 55)),
 1773: ((4, 5, 23, 28, 32), (6, 14, 30, 48, 50)),
 1774: ((14, 26, 30, 48, 50), (23, 27, 33, 55, 57)),
 1775: ((6, 16, 18, 21, 32), (2, 27, 30, 54, 55)),
 1776: ((2, 15, 27, 32, 48), (6, 16, 21, 46, 55)),
 1777: ((21, 40, 46, 55, 57), (10, 16, 23, 26, 32)),
 1778: ((23, 26, 32, 56, 57), (14, 16, 20, 30, 48)),
 1893: ((4, 10, 16, 32, 40), (21, 26, 42, 54, 56)),
 1895: ((4, 10, 29, 32, 55), (2, 14, 27, 33, 56)),
 1901: ((14, 

In [46]:
winner = comps[1762][0]
loser = comps[1762][1]
print(winner)
print(loser)

(2, 15, 18, 27, 55)
(14, 23, 33, 47, 48)


In [54]:
winner_previous_wins = 0
winner_previous_losses = 0
hero_counter = Counter()

for comp in comps:
    #hero_counter.update(comps[comp][0])
    #hero_counter.update(comps[comp][1])
    #print(comps[comp][0])
    if winner == comps[comp][0]:
        print(comps[comp][0])
        winner_previous_wins += 1
    if winner == comps[comp][1]:
        winner_previous_losses += 1
            


(2, 15, 18, 27, 55)


In [52]:
winner_previous_wins

1

In [53]:
winner_previous_losses

0

How many times was a hero (say, the first pick?) on the winning team?

In [60]:
print("winner: ", winner)
for comp in comps:
    #hero_counter.update(comps[comp][0])
    #hero_counter.update(comps[comp][1])
    #print(comps[comp][0])
    if winner[0] in comps[comp][0]:
        #print(comps[comp][0])
        if loser[0] in comps[comp][1]:
            print(comps[comp][0], " beat ", comps[comp][1])
        #winner_previous_wins += 1
    #if winner == comps[comp][1]:
    #    winner_previous_losses += 1

winner:  (2, 15, 18, 27, 55)
(2, 26, 30, 32, 48)  beat  (14, 19, 52, 53, 60) winner first pick beats loser first pick
(2, 15, 18, 27, 55)  beat  (14, 23, 33, 47, 48) winner first pick beats loser first pick
(2, 33, 42, 46, 60)  beat  (14, 22, 27, 55, 59) winner first pick beats loser first pick
(2, 50, 55, 57, 65)  beat  (12, 14, 23, 37, 47) winner first pick beats loser first pick
(2, 6, 27, 32, 33)  beat  (14, 20, 21, 26, 47) winner first pick beats loser first pick


In [61]:
for comp in comps:
    if winner[0] in comps[comp][1]:
        if loser[0] in comps[comp][0]:
            print(comps[comp][1], " lost to ", comps[comp][0])

(2, 11, 16, 48, 60)  lost to  (14, 27, 33, 51, 59)
(2, 6, 27, 32, 50)  lost to  (14, 23, 26, 30, 48)
(2, 38, 41, 42, 54)  lost to  (14, 21, 27, 40, 57)


When hero 2 and hero 14 face off, hero 2 has won 5 of 8 matches in HGC NA Phase 1.

In [65]:
for comp in comps:
    if winner[0] and winner[1] in comps[comp][0]:
        if loser[0] and loser[1] in comps[comp][1]:
            print(comps[comp][0], " beat ", comps[comp][1])

(2, 15, 18, 27, 55)  beat  (14, 23, 33, 47, 48)
(8, 15, 18, 31, 65)  beat  (23, 38, 47, 56, 63)
